In [40]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from numpy import sqrt
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import shapiro

In [41]:
intepr=pd.read_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Integrative proteomics\Integrative proteomics.xlsx", header=0, index_col="Gene names")
intepr

,Control,Control.1,ALS Sporadic,ALS Sporadic,ALS C9+,ALS C9+.1
Gene names,,,,,,
A1BG,360690.0,329620.0,751910.0,805240.0,774950.0,872080.0
A2M,4660400.0,4196000.0,6180900.0,6498000.0,5251800.0,5985500.0
AACS,316930.0,293370.0,304770.0,344310.0,273100.0,299120.0
AAGAB,99496.0,111270.0,115280.0,125580.0,99114.0,116670.0
AAK1,1150100.0,1128400.0,797970.0,871790.0,823370.0,833220.0
...,...,...,...,...,...,...
ZSWIM8,49895.0,54676.0,43025.0,58579.0,42613.0,48494.0
ZW10,553750.0,547250.0,604480.0,596200.0,530530.0,582260.0
ZYG11B,171070.0,163250.0,151230.0,157190.0,137830.0,143260.0


In [28]:
# Set the new range
min_range = 1
max_range = 5

In [ ]:
stat, p_value = shapiro(l17)

if p_value > 0.05:
    print("Data is normally distributed (fail to reject H0)")
else:
    print("Data is not normally distributed (reject H0)")

In [29]:
# Apply Min-Max scaling manually
intepr_min = intepr.min().min()
intepr_max = intepr.max().max()
scaled_intepr = (intepr - intepr_min) / (intepr_max - intepr_min)
scaled_datapr = (scaled_intepr * (max_range - min_range)) + min_range
scaled_datapr.head()

,Control,Control.1,ALS Sporadic,ALS Sporadic,ALS C9+,ALS C9+.1
Gene names,,,,,,
A1BG,1.000929,1.000849,1.001939,1.002076,1.001998,1.002249
A2M,1.012029,1.010830,1.015954,1.016773,1.013556,1.015450
AACS,1.000816,1.000755,1.000784,1.000886,1.000703,1.000770
AAGAB,1.000254,1.000285,1.000295,1.000322,1.000253,1.000299
AAK1,1.002967,1.002911,1.002058,1.002248,1.002123,1.002149


In [32]:
# Automatically identify group columns by their prefixes
als = [col for col in scaled_datapr.columns if col.startswith('ALS ')]
control = [col for col in scaled_datapr.columns if col.startswith('Control')]

# Perform t-tests row-wise
results = []
for i, row in scaled_datapr.iterrows():
    t_stat, p_value = ttest_ind(row[als].dropna(), row[control].dropna())
    results.append({'Gene names': i, 't_stat': t_stat, 'p_value': p_value})
    
# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df

C:\Users\Client\AppData\Local\Temp\ipykernel_24676\940665398.py:8: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat, p_value = ttest_ind(row[als].dropna(), row[control].dropna())


,Gene names,t_stat,p_value
0,A1BG,11.325735,0.000346
1,A2M,3.680742,0.021190
2,AACS,0.007538,0.994347
3,AAGAB,0.973457,0.385448
4,AAK1,-12.861174,0.000211
...,...,...,...
5492,ZSWIM8,-0.712577,0.515469
5493,ZW10,1.116169,0.326874
5494,ZYG11B,-2.890508,0.044540
5495,ZYX,-2.426986,0.072216


In [33]:
# Add the results DataFrame to the original data DataFrame
prot_stat = pd.concat([scaled_datapr, results_df.set_index('Gene names')], axis=1)
prot_stat.head()

,Control,Control.1,ALS Sporadic,ALS Sporadic,ALS C9+,ALS C9+.1,t_stat,p_value
Gene names,,,,,,,,
A1BG,1.000929,1.000849,1.001939,1.002076,1.001998,1.002249,11.325735,0.000346
A2M,1.012029,1.010830,1.015954,1.016773,1.013556,1.015450,3.680742,0.021190
AACS,1.000816,1.000755,1.000784,1.000886,1.000703,1.000770,0.007538,0.994347
AAGAB,1.000254,1.000285,1.000295,1.000322,1.000253,1.000299,0.973457,0.385448
AAK1,1.002967,1.002911,1.002058,1.002248,1.002123,1.002149,-12.861174,0.000211


In [34]:
# Apply Benjamini-Hochberg correction (optional)
_, p_adjusted, _, _ = multipletests(prot_stat['p_value'], method='fdr_bh')
prot_stat['p_adjusted'] = p_adjusted


In [35]:
_, p_adjusted, _, _ = multipletests(prot_stat['p_value'], method='bonferroni')
prot_stat['p_adjusted_bonf'] = p_adjusted
prot_stat.head()

,Control,Control.1,ALS Sporadic,ALS Sporadic,ALS C9+,ALS C9+.1,t_stat,p_value,p_adjusted,p_adjusted_bonf
Gene names,,,,,,,,,,
A1BG,1.000929,1.000849,1.001939,1.002076,1.001998,1.002249,11.325735,0.000346,0.009245,1.0
A2M,1.012029,1.010830,1.015954,1.016773,1.013556,1.015450,3.680742,0.021190,0.055468,1.0
AACS,1.000816,1.000755,1.000784,1.000886,1.000703,1.000770,0.007538,0.994347,0.996340,1.0
AAGAB,1.000254,1.000285,1.000295,1.000322,1.000253,1.000299,0.973457,0.385448,0.474750,1.0
AAK1,1.002967,1.002911,1.002058,1.002248,1.002123,1.002149,-12.861174,0.000211,0.008730,1.0


In [36]:
#df['average'] = df.mean(numeric_only=True, axis=1)
prot_stat['Mean Control'] = prot_stat.loc[:, prot_stat.columns.str.startswith('Control')].mean(axis=1)
prot_stat['Mean ALS'] = prot_stat.loc[:, prot_stat.columns.str.startswith('ALS ')].mean(axis=1)

In [37]:
#Fold Change
prot_stat["FC ALS-C"]= prot_stat["Mean ALS"]/prot_stat["Mean Control"]

In [38]:
#Log 2 FC
prot_stat["Log2FC ALS-C"]=np.log2(prot_stat["FC ALS-C"])
prot_stat.head()

,Control,Control.1,ALS Sporadic,ALS Sporadic,ALS C9+,ALS C9+.1,t_stat,p_value,p_adjusted,p_adjusted_bonf,Mean Control,Mean ALS,FC ALS-C,Log2FC ALS-C
Gene names,,,,,,,,,,,,,,
A1BG,1.000929,1.000849,1.001939,1.002076,1.001998,1.002249,11.325735,0.000346,0.009245,1.0,1.000889,1.002066,1.001176,1.695471e-03
A2M,1.012029,1.010830,1.015954,1.016773,1.013556,1.015450,3.680742,0.021190,0.055468,1.0,1.011430,1.015433,1.003959,5.699637e-03
AACS,1.000816,1.000755,1.000784,1.000886,1.000703,1.000770,0.007538,0.994347,0.996340,1.0,1.000785,1.000786,1.000000,6.512807e-07
AAGAB,1.000254,1.000285,1.000295,1.000322,1.000253,1.000299,0.973457,0.385448,0.474750,1.0,1.000270,1.000292,1.000023,3.268472e-05
AAK1,1.002967,1.002911,1.002058,1.002248,1.002123,1.002149,-12.861174,0.000211,0.008730,1.0,1.002939,1.002144,0.999208,-1.142993e-03


In [39]:
prot_stat.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Integrative proteomics\Integrative proteomics scaled T-test.xlsx")

In [19]:
fill=pd.read_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Integrative proteomics\Integrative proteomics scaled Mann-W.xlsx")
fill.head()

,Gene names,Uniprot,Protein family,Control,Control.1,ALS Sporadic,ALS Sporadic,ALS C9+,ALS C9+.1,U-stat,p_value,p_adjusted_bh,p_adjusted_bonf,Mean Control,Mean ALS,FC ALS-C,Log2FC ALS-C
0,A1BG,P04217,NaN,1.000929,1.000849,1.001939,1.002076,1.001998,1.002249,0.0,0.133333,0.207102,1,1.000889,1.002066,1.001176,1.695471e-03
1,A2M,P01023,Protease inhibitor I39 (alpha-2-macroglobulin) family,1.012029,1.010830,1.015954,1.016773,1.013556,1.015450,0.0,0.133333,0.207102,1,1.011430,1.015433,1.003959,5.699637e-03
2,AACS,Q86V21,ATP-dependent AMP-binding enzyme family,1.000816,1.000755,1.000784,1.000886,1.000703,1.000770,4.0,1.000000,1.000000,1,1.000785,1.000786,1.000000,6.512807e-07
3,AAGAB,Q6PD74,NaN,1.000254,1.000285,1.000295,1.000322,1.000253,1.000299,2.0,0.533333,0.615781,1,1.000270,1.000292,1.000023,3.268472e-05
4,AAK1,Q2M2I8,"Protein kinase superfamily, Ser/Thr protein kinase family",1.002967,1.002911,1.002058,1.002248,1.002123,1.002149,8.0,0.133333,0.207102,1,1.002939,1.002144,0.999208,-1.142993e-03


In [20]:
# Sort the DataFrame by the absolute values of the column
down_sorted = fill.loc[fill["Log2FC ALS-C"].abs().sort_values(ascending=False).index]

In [21]:
down_sorted.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Integrative proteomics\Integrative Mann-W sorted.xlsx")

In [23]:
# Filter rows where 'P-value' <= 0.05
filtered_down = down_sorted[down_sorted['p_value'] <= 0.05]

In [25]:
filtered_down.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\Integrative proteomics\Integrative Mann-W filtered.xlsx")